# Очистка Dataset

Dataset представляет из себя набор из картинок формата, к каждой из которых прилагается текстовый файл с соответствующими картинке тегами.
Загрузим все картинки и прикрепим к каждой из них теги

In [ ]:
from imutils import paths
import os
import shutil
import json
from tqdm import tqdm

In [ ]:
class DatasetImage:
    def __init__(self, tagsPath):
        self.tagsPath = tagsPath
        self.imagePath = tagsPath[:-4] + '.jpg'
        self.id = int(tagsPath.split(os.sep)[-1][:-4])
        self.tags = set()
        with open(tagsPath) as file:
            rawTagLines = file.readlines()
            for tagLine in rawTagLines:
                for tag in tagLine.split(','):
                    self.tags |= {tag.strip()}

Узнаем, какие у нас расширения встречаются в Dataset. Ожидаем, что мы увидим только .txt и .jpg

In [ ]:
extensions = set()

for file in paths.list_files("./dataset"):
    extensions.add(file.split('.')[-1])

extensions

{'jpg', 'txt'}

Все файлы являются либо .txt, либо .jpg файлами. OK

Подсчитаем количество .txt и .jpg файлов. Их должно быть одинаковое количество

In [ ]:
extDict = { 'jpg': 0, 'txt': 0 }

for file in paths.list_files("./dataset"):
    fileExt = file.split('.')[-1]
    extDict[fileExt] += 1

extDict

{'jpg': 30450, 'txt': 30450}

Количество совпадает. ОК

Загрузим изображения

In [ ]:
images = []

for file in tqdm(paths.list_files("./dataset"), 'Loading Images'):
    if file.endswith('.txt'):
        img = DatasetImage(file)
        images.append(img)

len(images)

Посмотрим встречающиеся теги

In [ ]:
def show_tags(images):
    allTagsSet = set()

    for image in images:
        allTagsSet |= set(image.tags)

    tagsDict = {x: 0 for x in list(allTagsSet)}

    for img in images:
        for tag in img.tags:
            tagsDict[tag] += 1

    sortedTagsDictItems = sorted(tagsDict.items(), key=lambda item: item[1])
    sortedTagsDictItems.reverse()

    sortedTagsDict = dict(sortedTagsDictItems)

    for key, value in sortedTagsDict.items():
        print(f'{key}: {value}')

Выведем список всех тегов

In [ ]:
show_tags(images)

### Функции для обработки

In [ ]:
class MappedTouhouImage:
    def __init__(self, image, character_name):
        self.id = image.id
        self.character_name = character_name
        self.imagePath = image.imagePath
        self.tags = image.tags
        self.tagsPath = image.tagsPath

In [ ]:
class PersonStats:
    def __init__(self):
        self.other = 0
        self.solo = 0
        self.main = 0

In [ ]:
def copy_images_with_tags_to(images, selected_tags, folder_name):
    selected_images = []

    new_dir_path = os.path.join('./temp', folder_name)
    if not os.path.exists(new_dir_path):
        os.mkdir(new_dir_path)

    for image in images:
        if len(image.tags & selected_tags) != 0:
            selected_images.append(image)

    if (len(selected_images) == 0):
        print("Nothing to copy")
        return images

    movedImages = []

    for image in tqdm(selected_images, f'Copying images to {folder_name}'):
        newGenericPath = os.path.join(new_dir_path, str(image.id))

        if (not os.path.exists(image.imagePath)):
            print(f'WARNING! IMAGE {image.imagePath} DOES NOT EXISTS!')
            continue

        if (not os.path.exists(image.imagePath)):
            print(f'WARNING! IMAGE EXISTS BUT TAGS {image.tagsPath} DOES NOT EXISTS!')
            continue

        shutil.copy(image.imagePath, newGenericPath + '.jpg')
        shutil.copy(image.tagsPath, newGenericPath + '.txt')

        movedImages.append(DatasetImage(newGenericPath + '.txt'))

    return movedImages

In [ ]:
def move_images_with_tags_to(images, selected_tags, folder_name):
    selected_images = []

    new_dir_path = os.path.join('./temp', folder_name)
    if not os.path.exists(new_dir_path):
        os.mkdir(new_dir_path)

    for image in images:
        if len(image.tags & selected_tags) != 0:
            selected_images.append(image)

    if (len(selected_images) == 0):
        print("Nothing to move")
        return [], images

    movedImages = []
    clearedImages = images.copy()

    for image in tqdm(selected_images, f'Moving images to {folder_name}'):
        newGenericPath = os.path.join(new_dir_path, str(image.id))

        if (not os.path.exists(image.imagePath)):
            print(f'WARNING! IMAGE {image.imagePath} DOES NOT EXISTS!')
            continue

        if (not os.path.exists(image.imagePath)):
            print(f'WARNING! IMAGE EXISTS BUT TAGS {image.tagsPath} DOES NOT EXISTS!')
            continue

        shutil.move(image.imagePath, newGenericPath + '.jpg')
        shutil.move(image.tagsPath, newGenericPath + '.txt')

        movedImages.append(DatasetImage(newGenericPath + '.txt'))
        clearedImages.remove(image)

    return movedImages, clearedImages

In [ ]:
def move_images_to_final(movingImages, folder_name):
    new_dir_path = os.path.join('./mapped_dataset', folder_name)
    if not os.path.exists(new_dir_path):
        os.mkdir(new_dir_path)

    if (len(movingImages) == 0):
        print("Nothing to move")
        return [], movingImages

    movedImages = []

    for image in tqdm(movingImages, f'Moving images to final: {folder_name}'):
        newGenericPath = os.path.join(new_dir_path, str(image.id))

        if (not os.path.exists(image.imagePath)):
            print(f'WARNING! IMAGE {image.imagePath} DOES NOT EXISTS!')
            continue

        if (not os.path.exists(image.imagePath)):
            print(f'WARNING! IMAGE EXISTS BUT TAGS {image.tagsPath} DOES NOT EXISTS!')
            continue

        shutil.move(image.imagePath, newGenericPath + '.jpg')
        shutil.move(image.tagsPath, newGenericPath + '.txt')

        movedImages.append(DatasetImage(newGenericPath + '.txt'))

    return movedImages

In [ ]:
cleared_images = []
moved_images = []
persons_img_count = dict()

### Обработка

#### Процесс обработки:
1. Выбираем любого персонажа из [Touhou Wiki](https://touhou.fandom.com/wiki/Category:Characters)
2. Ищем теги, которые соответствуют этому персонажу. Как правило их 1-2. Добавлем эти теги в следущем формате:
```
[имя_персонажа]: {
    [тег1],
    [тег2]
}
```
`имя_персонажа` в данном случае это название директории, куда будут сгружаться картинки.

3. Выполняем все ячейки до конца.

#### Что происходит при обработке:
1. Все найденные с тегами персонажа изображения копируются в директорию `./temp/[имя_персонажа]`
2. Отфильтровываются изображения, которые нам точно не понадобятся. Это:
- NSFW контент;
- Изображения персонажа, которые сильно отличаются от его типичного стиля

Такие изображения отправляются в `./temp/[имя_персонажа]/other`

3. Останутся картинки, на которых может быть несколько персонажей сразу. Это не так плохо, но лучше всего будет отфильтровать картинки, где есть только рассматриваемый персонаж. Поэтому отправляем в отдельную директорию `./temp/[имя_персонажа]/solo` все картинки с тегами `solo`, `1girl`. 

4. Картинки из `./temp/[имя_персонажа]/solo` перемещаем в `./mapped_dataset/[имя_персонажа]`

In [ ]:
show_tags(images)

In [ ]:
persons = {
    'alice_margatroid': {
        'alice margatroid (pc-98)',
        'alice margatroid'
    }

    'flandre_scarlet': {
        'flandre scarlet',
        'flandre scarlet (vampire pursuing the hunter)'
    },

    'hakurei_reimu': {
        'hakurei reimu',
        'hakurei reimu (pc-98)',
    },

    'remilia_scarlet': {
        'remilia scarlet',
    },

    'ibuki_suika': {
        'ibuki suika',        
    },

    'yakumo_yukari': {
        'yakumo yukari',        
    },

    'komeiji_satori': {
        'komeiji satori',
        'foul detective satori',
    },

    'konpaku_youmu': {
        'konpaku youmu',
        'konpaku youmu (ghost)'
    },

    'komeiji_koishi': {
        'komeiji koishi',
        'koishi day'
    },

    'kirisame_marisa': {
        'kirisame marisa',
        'kirisame marisa (pc-98)',
    },

    'izayoi_sakuya': {
        'izayoi sakuya'
    },

    'rumia': {
        'rumia'
    }
}

In [ ]:
person_folder_name = ''

In [ ]:
def initial_copy():
    if (not person_folder_name in persons_img_count):
        persons_img_count[person_folder_name] = PersonStats()

    selected_tags = persons[person_folder_name]

    cleared_images = copy_images_with_tags_to(images, selected_tags, person_folder_name)

In [ ]:
def filter_nsfw_other():
    selected_tags = { 
        'tentacles',
        'yuri',
        'nude',
        'cosplay'
        'sex', 
        'group sex',
        'vaginal',
        'penis',
        'ass', 
        'used condom', 
        'spread legs', 
        'imminent vaginal',
        'imminent rape',
        'masturbation',
        'nipples',
        'huge breasts', 
        '1boy',
        'cum',
        'bikini',
        'skirt lift',
        'lifted by self',
        'underwear',
        'underwear only'
        'alternate costume',
        'santa costume',
        'swimsuit',
        'loli',
        'toes',
        'cosplay',
        'animal ears',
        'rabbit ears',
        'animal ears', }

    moved_images, cleared_images = move_images_with_tags_to(cleared_images, selected_tags, f'{person_folder_name}\\other')
    persons_img_count[person_folder_name].other = len(moved_images)

In [ ]:
def filter_solo():
    selected_tags = {
        'solo',
        '1girl'
    }

    moved_images, cleared_images = move_images_with_tags_to(cleared_images, selected_tags, f'{person_folder_name}\\solo')
    persons_img_count[person_folder_name].solo = len(moved_images)
    persons_img_count[person_folder_name].main = len(cleared_images)

    
    move_images_to_final(moved_images, person_folder_name)

In [ ]:
def print_stats():
    sortedPersonsDictItems = sorted(persons_img_count.items(), key=lambda item: item[1].solo + item[1].main + item[1].other)
    sortedPersonsDictItems.reverse()

    persons_img_count = dict(sortedPersonsDictItems)

    for key, value in persons_img_count.items():
        print(f'{key:20} - solo: {value.solo:4}, main: {value.main:4}, nsfw/other: {value.other:4}')

    classes_count = len(persons)
    total_images = sum(value.main + value.solo + value.other for key, value in persons_img_count.items())
    print(f'Total images: {total_images}')
    print(f'Total classes: {classes_count}')

In [ ]:
for person, tags in persons.items():
    person_folder_name = person

    initial_copy()
    filter_nsfw_other()
    filter_solo()

print_stats()

Moving images to final: rumia: 100%|██████████| 217/217 [00:00<00:00, 238.72it/s]

flandre_scarlet      - solo: 1338, main:  330, nsfw/other:  552
hakurei_reimu        - solo: 1301, main:  380, nsfw/other:  492
komeiji_koishi       - solo: 1415, main:  346, nsfw/other:  374
kirisame_marisa      - solo:  964, main:  363, nsfw/other:  368
konpaku_youmu        - solo:  969, main:   89, nsfw/other:  328
remilia_scarlet      - solo:  814, main:  265, nsfw/other:  259
izayoi_sakuya        - solo:  551, main:  109, nsfw/other:  300
komeiji_satori       - solo:  433, main:  249, nsfw/other:  205
yakumo_yukari        - solo:  309, main:  115, nsfw/other:  172
alice_margatroid     - solo:  266, main:  101, nsfw/other:  156
rumia                - solo:  217, main:   16, nsfw/other:   65
ibuki_suika          - solo:  120, main:   35, nsfw/other:   40
Total images: 14406
Total classes: 12
